This code is built while following a youtube video by Keerti Purswani. 

Although this is a follow along code book, it contains my notes and some modifications.


## Langgraph
Langgrpah uses a graph-based approach to handle complex and dynamic workflows for building LLM based applications.

Some imports and followed by defining the state of the graph.

In [2]:
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate

In [3]:
llm = ChatOllama(model = "qwen2.5:7b") # loading up the model that is available.

### Building a Recruitment Agency Workflow

A Simple worflow that either selects an applicant if he matches the requirement or forwards for further review if the applicant is a senior-level categorization.



In [4]:
class State(TypedDict):
    experience: str
    application: str
    skill_match: str
    response: str

In [28]:
def categorizing_experience(state):
    prompt = ChatPromptTemplate.from_template(
        "You are a classification assistant. Based only on the content below, classify the experience level of the application as one of the following three categories:\n"
        "- Entry-level\n"
        "- Mid-level\n"
        "- Senior-level\n\n"
        "Respond with exactly one of the above options and nothing else.\n\n"
        "Application:\n{application}"
    )

    chain = prompt | llm
    experience_level = chain.invoke({"application": state["application"]}).content.strip()

    return {"experience": experience_level}



In [37]:
def assistant_to_assess(state):
    prompt = ChatPromptTemplate.from_template("Based on the job application for a C++ developer, assess the candidate's skillset." \
    "Respond strictly with either a 'Match' or 'No Match'." \
    "Application: {application}" \
    "reply only with either 'Match' or 'No Match'" \
    )

    chain = prompt|llm
    skill_match = chain.invoke({"application": state["application"]}).content

    return {"skill_match": skill_match}

def hr_interview(state):
    return {"response": "Shortlisted for HR Interview"}

def rejection(state):
    return {"response": "Sorry to inform, rejected"}

def recruiter_to_review(state):
    return {"response": "Needs further review"}


In [38]:
def route(state):
    if(state["skill_match"] == "Match"):
        return "hr_interview"
    elif state["experience"] == "Senior-level":
        return "recruiter_to_review"
    else:
        return "rejection"

In [39]:
builder = StateGraph(State)

# defining the nodes of the graphs
builder.add_node("categorizing_experience", categorizing_experience)
builder.add_node("assistant_to_assess", assistant_to_assess)
builder.add_node("rejection", rejection)
builder.add_node("recruiter_to_review", recruiter_to_review)
builder.add_node("hr_interview",hr_interview)

# defining the edges of the graph
builder.add_edge(START, "categorizing_experience")
builder.add_edge("categorizing_experience", "assistant_to_assess")
builder.add_conditional_edges("assistant_to_assess", route)
builder.add_edge("recruiter_to_review", END)
builder.add_edge("hr_interview", END)
builder.add_edge("recruiter_to_review", END)

graph = builder.compile()

# Image(graph.get_graph().draw_mermaid_png())

In [40]:
application = "I have a 15 years experience in Python" 
response = graph.invoke({"application": application})
print(response["response"])

Needs further review
